In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
import os
import matplotlib as plt
# Import the API key.
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/city_weather.csv")
city_data_df.head()

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Cloudiness,Wind Speed
0,Ituiutaba,BR,1638313104,89,-18.9689,-49.4650,73.20,100,3.87
1,Rengāli,IN,1638313107,59,21.6333,84.0500,57.87,2,4.83
2,East London,ZA,1638313109,81,-33.0153,27.9116,66.69,84,5.73
3,Laas,IT,1638313112,93,46.6166,10.7002,43.54,63,7.49
4,Saint Paul Harbor,US,1638313114,81,57.7900,-152.4072,27.55,89,5.55


In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)



In [4]:
# Using Lat and Lng as locations:
locations = city_data_df[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_data_df["Humidity"].astype(float)

In [5]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 75) & \
                                       (city_data_df["Max Temp"] >= 90)]
preferred_cities_df.head(10)

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Cloudiness,Wind Speed


In [16]:
# Create a hotel_df
hotel_df = preferred_cities_df.loc[:,["City","Country", "Max Temp", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name


In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key
          "location" : "48.8566, 2.3522"
         }

SyntaxError: invalid syntax (3116784643.py, line 7)

In [20]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Hotels not found... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

-------End of Search-------


In [22]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

In [23]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [24]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig